In [3]:
# 制度文件语料预处理实践
# 公文是党政机关实施领导、履行职能等的具有特定效力和规范体式的文书，是传达贯彻党和国家的方针政策，公布法规和规章等的重要工具。
# 其中，正文是公文的主体，用来表述公文的内容；附件是公文正文的说明、补充或者参考资料。
# 规章制度文件是公文的一种形式。一般来说，对于制度文件发布，多采用“通知”这一公文文种。因此，规章制度是规范性文件，令其文本内容作为语料，需重点关注其所属公文的附件
# 而所属公文的正文，包含发文机关标志、发文字号、发文机关署名、成文日期等，可作为规章制度文件的元数据。本阶段关注规章制度文件文本内容的预处理。
# 目前，气政通信息中心“制度区”发布的规章制度文件，主要有docx、doc、pdf三种格式。开展制度文件语料制作，首要需将这三种格式文件进行文本解析，获取标准字符串
# 一般可统一为txt或markdown格式文件存储,因原始制度文件文本存在标题级别，本文选择markdown格式。


# DOCX文件转换（全部）
from docx import Document
def docx_to_text(file_path):
    doc = Document(file_path)
    text = ""
    for para in doc.paragraphs:
        text += para.text + "\n\n"
    return text

# 转换文本为Markdown格式
def text_to_markdown_with_titles(text):
    markdown = ""
    markdown += text
    return markdown


# 使用函数并将结果写入新文件
docx_text = docx_to_text('E:/code/documentProcess/doc/example.docx')
markdown_docx = text_to_markdown_with_titles(docx_text)
# 将Markdown文本写入文件
with open('E:/code/documentProcess/output/docx_to_markdown1.md', 'w', encoding='utf-8') as md_file:
    md_file.write(markdown_docx)
    
# 最终效果：无表格、图片，只有正文文本 

In [3]:
# DOC文件转换（全部）

import os

# 文件当前路径
current_file_path = 'E:/code/documentProcess/doc/example.doc'
# 新的文件路径，包括新的文件名和后缀
new_file_path = 'E:/code/documentProcess/doc/exampleFromDoc.docx'

# 重命名文件
if not os.path.exists(new_file_path):
    os.rename(current_file_path, new_file_path)
else:
    print(f"文件 {new_file_path} 已存在，不会进行重命名。")

In [ ]:
# PDF文件转换-第一步(方法1)

from langchain.document_loaders import UnstructuredPDFLoader

loader = UnstructuredPDFLoader('E:/code/documentProcess/doc/example.pdf')
data = loader.load()
# 数据的形态
# print(data[0].page_content)
txt_file = open(
    "E:/code/documentProcess/output/pdf_to_md_langchain.md", mode='a', encoding='utf-8')
txt_file.write(data[0].page_content)

In [27]:
# PDF文件转换-第一步(方法2)

# 最常用的两个库「pdfplumber」、「pypdf2」
# pdfplumber库按页处理 pdf ，获取页面文字，提取表格等操作。
# PyPDF2 是一个纯 Python PDF 库，可以读取文档信息（标题，作者等）、写入、分割、合并PDF文档，它还可以对pdf文档进行添加水印、加密解密等。
import PyPDF2

# PDF文件转换为文本
def pdf_to_text(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfFileReader(file)
        text = ""
        for page_num in range(reader.numPages):
            page = reader.getPage(page_num)
            text += page.extractText() + "\n\n"
        return text

# 转换文本为Markdown格式
def text_to_markdown_with_titles(text):
    markdown = ""
    markdown += text
    return markdown


# 使用函数并将结果写入新文件
pdf_text2 = pdf_to_text('E:/code/documentProcess/doc/example.pdf')
markdown_pdf = text_to_markdown_with_titles(pdf_text2)


# 将Markdown文本写入文件
with open('E:/code/documentProcess/output/pdf_to_md_PyPDF2.md', 'w', encoding='utf-8') as md_file:
    md_file.write(markdown_pdf)

In [7]:
# 文本清洗，合并一个完整句子

# 打开Markdown文件进行读取
path1="E:/code/documentProcess/output/pdf_to_md_PyPDF2.md"
with open(path1, 'r',
          encoding='utf-8') as file:
    lines = file.readlines()

# 遍历每一行，进行处理
new_lines = []
last_line = ''
for line in lines:
    line = line.strip()  # 去除行首尾的空格或换行符
    if line and (line.endswith('。')):
        new_lines.append(last_line.strip() + ' ' + line)  # 如果以句号或分号结尾，添加到新行中
        last_line = ''
    else:
        last_line = last_line.strip() + ' ' + line if last_line else line
        # 否则与上一行合并

# 如果最后一行不为空，添加到新行列表中
if last_line:
    new_lines.append(last_line)

# 将处理后的内容写回文件
with open('E:/code/documentProcess/output/pdf2md_merged.md', 'w', encoding='utf-8') as file:
    file.write('\n'.join(new_lines))

print("文件处理完成。")

文件处理完成。


In [15]:
# 文本清洗，删除页码

import re

# 打开文件，读取内容
with open('E:/code/documentProcess/output/pdf2md_merged.md', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# 允许在数字前后只有一个或两个空格  
pattern = r'—\s{1,2}\d+\s{1,2}—'  
# 使用正则表达式替换包含特定数字的行
new_lines = [re.sub(pattern, '', line) for line in lines]

# 写入到文件中
with open('E:/code/documentProcess/output/pdf2md_merged_DelpageNum.md', 'w', encoding='utf-8') as file:
    file.writelines(new_lines)

In [16]:
# 文本清洗，删除每一行的空格

# 打开文件，读取内容
with open('E:/code/documentProcess/output/pdf2md_merged_DelpageNum.md', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# 删除每一行中的空格
new_lines = [line.replace(' ', '') for line in lines]

# 写入到文件中
with open('E:/code/documentProcess/output/pdf2md_merged_Delspace.md', 'w', encoding='utf-8') as file:
    file.writelines(new_lines)